建立Dataset

In [44]:
from torch.utils.data import Dataset
import pandas as pd #for read data
import torch
class MyDemoDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        data = pd.read_csv("~/Desktop/train.csv")
        self.x = data[["title1_zh", "title2_zh"]]
        self.y = data["label"]
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        #無意義處理，純粹示範用
        result = [len(self.x.iloc[index, 0]), len(self.x.iloc[index, 1])]
        return result

建立Dataloader

In [45]:
from torch.utils.data import DataLoader
myDataLoader = DataLoader(dataset=MyDemoDataset(), shuffle=True, batch_size=5)

In [41]:

for i, data in enumerate(myDataLoader):
    print(data)
    if i == 3:
        break 
    
demo_iter = iter(myDataLoader)
next(demo_iter)

[tensor([21, 29, 23, 28, 23]), tensor([35, 30, 19, 30, 30])]
[tensor([13, 30, 25, 15, 22]), tensor([22, 28, 29, 27, 25])]
[tensor([27, 15, 26, 23, 10]), tensor([32, 19, 22, 32, 17])]
[tensor([25, 32, 28, 19, 13]), tensor([22, 32, 23, 28, 30])]


Transformers示範:
Pre-trained Tokenizer from BERT Model

In [43]:
from transformers import BertTokenizerFast
MY_PRETRAINED_MODEL = "bert-base-chinese"
tokenizer = BertTokenizerFast.from_pretrained(MY_PRETRAINED_MODEL)
tokenizer()

[tensor([20, 22, 25, 17, 30]), tensor([26, 15, 30, 26, 26])]

In [ ]:
tmp.columns
len(tmp)
import os
os.cpu_count()